<a href="https://colab.research.google.com/github/ericyoc/key_exchanges_demo_poc/blob/main/key_exchanges_demo_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install prettytable

In [2]:
import random
from prettytable import PrettyTable

In [3]:
# Diffie-Hellman Key Exchange functions
def generate_dh_prime():
    return 23  # Small prime for demonstration

def generate_base(prime):
    return 5  # Should be a primitive root modulo p in practice

def generate_private_key(prime):
    return random.randint(1, prime - 1)

def compute_public_key(prime, base, private_key):
    return pow(base, private_key, prime)

def compute_shared_secret(prime, public_key, private_key):
    return pow(public_key, private_key, prime)

In [4]:

# RSA functions
def generate_rsa_primes():
    return 11, 17  # Small primes for demonstration

def mod_inverse(e, phi):
    for d in range(3, phi):
        if (d * e) % phi == 1:
            return d
    raise ValueError("mod_inverse does not exist")

def generate_rsa_keypair():
    p, q = generate_rsa_primes()
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 7  # Common choice for small examples
    d = mod_inverse(e, phi)
    return ((e, n), (d, n))

def rsa_encrypt(public_key, plaintext):
    e, n = public_key
    return pow(plaintext, e, n)

def rsa_decrypt(private_key, ciphertext):
    d, n = private_key
    return pow(ciphertext, d, n)

def rsa_sign(private_key, message):
    return rsa_encrypt(private_key, message)  # In practice, we'd hash the message first

def rsa_verify(public_key, message, signature):
    return rsa_decrypt(public_key, signature) == message

In [5]:
def demonstrate_scenarios():
    # Scenario 1: Diffie-Hellman Key Exchange
    print("Scenario 1: Diffie-Hellman Key Exchange")
    prime = generate_dh_prime()
    base = generate_base(prime)

    alice_private = generate_private_key(prime)
    alice_public = compute_public_key(prime, base, alice_private)

    bob_private = generate_private_key(prime)
    bob_public = compute_public_key(prime, base, bob_private)

    alice_shared = compute_shared_secret(prime, bob_public, alice_private)
    bob_shared = compute_shared_secret(prime, alice_public, bob_private)

    table = PrettyTable()
    table.title = "Diffie-Hellman Key Exchange"
    table.field_names = ["Step", "Alice", "Bob", "Eve"]
    table.add_row(["1. Agree on parameters", f"p = {prime}, g = {base}", f"p = {prime}, g = {base}", f"Observes p = {prime}, g = {base}"])
    table.add_row(["2. Generate private key", f"a = {alice_private}", f"b = {bob_private}", "Cannot know a or b"])
    table.add_row(["3. Compute public key", f"A = g^a mod p = {alice_public}", f"B = g^b mod p = {bob_public}", f"Observes A = {alice_public}, B = {bob_public}"])
    table.add_row(["4. Exchange public keys", "Sends A to Bob", "Sends B to Alice", "Intercepts A and B"])
    table.add_row(["5. Compute shared secret", f"s = B^a mod p = {alice_shared}", f"s = A^b mod p = {bob_shared}", "Cannot compute s"])
    print(table)

    # Scenario 2: RSA Encryption and Decryption
    print("\nScenario 2: RSA Encryption and Decryption")
    alice_public, alice_private = generate_rsa_keypair()
    message = 42  # Small number for demonstration
    encrypted = rsa_encrypt(alice_public, message)
    decrypted = rsa_decrypt(alice_private, encrypted)

    table = PrettyTable()
    table.title = "RSA Encryption and Decryption"
    table.field_names = ["Step", "Alice", "Bob", "Eve"]
    table.add_row(["1. Generate key pair", f"Public: {alice_public}\nPrivate: {alice_private}", "Not involved", "Observes public key"])
    table.add_row(["2. Share public key", "Sends public key to Bob", "Receives Alice's public key", "Observes public key exchange"])
    table.add_row(["3. Prepare message", "Not involved", f"m = {message}", "Unknown"])
    table.add_row(["4. Encrypt message", "Not involved", f"c = m^e mod n = {encrypted}", f"Observes c = {encrypted}"])
    table.add_row(["5. Send encrypted message", "Receives c", "Sends c to Alice", "Intercepts c"])
    table.add_row(["6. Decrypt message", f"m = c^d mod n = {decrypted}", "Not involved", "Cannot decrypt"])
    print(table)

    # Scenario 3: RSA Signing
    print("\nScenario 3: RSA Signing")
    message_to_sign = 42  # Small number for demonstration
    signature = rsa_sign(alice_private, message_to_sign)
    is_valid = rsa_verify(alice_public, message_to_sign, signature)

    table = PrettyTable()
    table.title = "RSA Signing"
    table.field_names = ["Step", "Alice", "Bob", "Eve"]
    table.add_row(["1. Prepare message", f"m = {message_to_sign}", "Not involved", "Unknown"])
    table.add_row(["2. Sign message", f"s = m^d mod n = {signature}", "Not involved", "Cannot sign"])
    table.add_row(["3. Send message and signature", "Sends m and s to Bob", "Receives m and s", "Intercepts m and s"])
    table.add_row(["4. Verify signature", "Not involved", f"m == s^e mod n: {is_valid}", "Can verify, cannot forge"])
    print(table)

In [6]:
if __name__ == "__main__":
    demonstrate_scenarios()

Scenario 1: Diffie-Hellman Key Exchange
+--------------------------------------------------------------------------------------------+
|                                Diffie-Hellman Key Exchange                                 |
+--------------------------+-------------------+--------------------+------------------------+
|           Step           |       Alice       |        Bob         |          Eve           |
+--------------------------+-------------------+--------------------+------------------------+
|  1. Agree on parameters  |   p = 23, g = 5   |   p = 23, g = 5    | Observes p = 23, g = 5 |
| 2. Generate private key  |       a = 18      |       b = 21       |   Cannot know a or b   |
|  3. Compute public key   | A = g^a mod p = 6 | B = g^b mod p = 14 | Observes A = 6, B = 14 |
| 4. Exchange public keys  |   Sends A to Bob  |  Sends B to Alice  |   Intercepts A and B   |
| 5. Compute shared secret | s = B^a mod p = 4 | s = A^b mod p = 4  |    Cannot compute s    |
+---------